In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Math
from matplotlib.font_manager import FontProperties
from matplotlib import rc
import matplotlib as mpl
import sys
sys.path.append("..")
from exotop import topo_1D_plots as plottop
from exotop.model_1D import inputs as ins
from exotop.model_1D import parameters as p

# set paths
# fig_path = '/space/cmg76/figs/' # office
fig_path = 'figs/' # laptop#
fig_format = '.png'
benchmark_path = '../benchmarks/'

# set mpl display parameters
mpl.rcParams['axes.linewidth'] = 2  # set the value globally - for poster
rc('font',**{'family':'serif','serif':['Times New Roman']})  # latex font for matplotlib CMU Serif Roman
# rc('font',**{'family':'monospace','monospace':['Ubuntu Mono']})  # for computery poster 
rc('text', usetex=True)  # turn on for latexy font
mpl.font_manager._rebuild()
plt.style.use('dark_background')